In [1]:
#Definindo finais de semana
mes = "07"
lista_fds = ['06','07','13','14','20','21','27','28','31']

# Funções e Pacotes

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#definindo quantidade de linhas e colunas visiveis
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Manipulação de Strings
import unidecode # => unidecode(string)
import re # Regex

# Repositorio - Diretorio
import os
import shutil

In [3]:
#######################-FUNÇÕES DE CONTAGEM
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)
        
def tabela_cont(linha, coluna):
    a = df.groupby([linha, coluna]).size().unstack()
    for i in a.columns:
        a[i].fillna(0, inplace = True)
        a[i] = a[i].apply(lambda x: int(x))
    return a
    print('='*100)

def tabela_cont_percent(linha, coluna, opcao):
    result = tabela_cont(linha, coluna)
    if opcao == 'Linha':
        return round(result.div(result.sum(axis=1), axis=0) * 100, 2)
    elif opcao == 'Coluna':
        return round(result.div(result.sum(), axis=1) * 100, 2)
    else:
        return 'Opção invalida'

In [4]:
# Funções de salvar arquivo
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

In [5]:
def read_file(file_path):
    # 1. Ler o arquivo txt
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    # 2. Remover todas as linhas que contêm "Mensagem apagada"
    lines = [line for line in lines if "Mensagem apagada" not in line]
    
    # 3. Corrigir as mensagens que estão em várias linhas para que fiquem em uma única linha
    corrected_lines = []
    current_message = ""
    
    for line in lines:
        # Se a linha começa com o padrão de data e hora, ela é uma nova mensagem
        if re.match(r"^\[\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}\]", line):
            if current_message:  # Se há uma mensagem anterior, adicione-a à lista
                corrected_lines.append(current_message)
            current_message = line.strip()  # Comece uma nova mensagem
        else:
            current_message += " " + line.strip()  # Adicione o conteúdo da linha atual à mensagem em progresso
    
    # Adiciona a última mensagem, se houver
    if current_message:
        corrected_lines.append(current_message)
    
    # Salvando arquivo txt
    with open('processed.txt', "w", encoding="utf-8") as file:
        for line in corrected_lines:
            file.write(line + "\n")
    
    # Lendo arquivo txt e transformando em dataframe cada mensagem
    with open('processed.txt', 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    
    return content

In [6]:
def sample_prob(nivel):
    '''
    data = df_prob[df_prob['controle'] == nivel]
    tamanho = len(data)
    try:
        if tamanho == 0:
            data_exibir = str('0 itens do prob' + nivel)
        if tamanho == 1:
            data_exibir = data['mensagem'].sample(1)
        if tamanho == 2:
            data_exibir = data['mensagem'].sample(2)
        if tamanho > 5:
            data_exibir = data['mensagem'].sample(5)
    except:
        data_exibir = 'erro'
    '''
    return 'nada'

# =========================================

# Carregando base

In [7]:
#df = pd.DataFrame({'chat': read_file('_chat.txt') })
df = pd.read_excel('df_prob.xlsx')[['chat']]; os.remove("df_prob.xlsx")

#df['chat'] = df['chat'].apply(lambda x: x.replace('/05/2024, ','/05/2024 '))

df['controle'] = np.nan
df = df[['controle','chat']]
df_prob = pd.DataFrame(columns = ['controle','chat'])

print(len(df))
display(df.head())

265


,controle,chat
0,NaN,[05/07/2024 08:09:10] Alessandre Luiz Braga Ju...
1,NaN,[01/07/2024 07:42:01] Alexandre Amaral Lucas: ...
2,NaN,[01/07/2024 23:51:04] Alexandre Amaral Lucas: ...
3,NaN,[15/07/2024 19:01:44] Alexandre Amaral Lucas: ...
4,NaN,[03/07/2024 01:40:52] Alexandre Bernardino: 03...


,controle,chat


# Dia de envio da mensagem

In [8]:
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
#print(sorted(df['dia'].unique()))
print("Quantidade de mensagens por dia:")
count_freq(df, 'dia')

Quantidade de mensagens por dia:


,Freq,Perc
dia,,
07/07/2024,34,12.830189
14/07/2024,25,9.433962
08/07/2024,23,8.679245
03/07/2024,18,6.792453
09/07/2024,17,6.415094
12/07/2024,17,6.415094
05/07/2024,16,6.037736
02/07/2024,15,5.660377
11/07/2024,15,5.660377


# Nome Anestesista

In [9]:
df['nome'] = np.nan
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome = df['chat'][i][loc+1:loc+loc2]
    nome = nome.upper().strip().replace("  ", " ")
    nome = unidecode.unidecode(nome)
    df['nome'][i] = nome
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))

print("Profissionais que mandaram mensagem:")
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))
sorted(df['nome'].unique())

Profissionais que mandaram mensagem:


['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURAO BARRETO',
 'ANA JUNG DOCKO',
 'ANDRE AUGUSTO BELLINI',
 'ANDRE DA COSTA AGUIAR VENTURA',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'CAIO TULIO DUARTE',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO MUELLER KAKUDA',
 'DANIEL MASUZAKI WONG',
 'DEBORA PHILIPPI BRESSANE',
 'EDUARDO ARBACHE BEZERRA',
 'EDUARDO DALA ROSA SOARES',
 'EMERSON DOMINGOS DA COSTA',
 'ESTANRLEY BARCELOS PINTO',
 'FABIANA BARBOSA BASTOS',
 'FABIANA SOARES GAMA',
 'FABIO ANTONIO DE SOUZA OLIVEIRA',
 'FABIO YUJI SUGUITA',
 'FELIPE DE OLI

# Mensagens escrevidas e enviadas

In [10]:
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'] = df['mensagem'].apply(lambda x: x.upper() )
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('-',' - '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))

print('Mensagens Enviadas:')
df['mensagem'].sample(3)

Mensagens Enviadas:


190                     04/07 OSASCO EMERSON SAÍDA 18:40
47     14/07 MORUMBI  -  FLAVIO ENTRADA 06:40  ( D2 R...
217                  10/07 JABAQUARA RIBEIRO SAÍDA 16:45
Name: mensagem, dtype: object

# =========================================

# Data do Plantão

In [11]:
df['data'] = np.nan

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'DATA_DO_PLANTÃO'

#salvando
df_prob = df[df['controle'] == 'DATA_DO_PLANTÃO']
print("data probs:", len(df_prob))
sample_prob('DATA_DO_PLANTÃO')

df = df[df['controle'] != 'DATA_DO_PLANTÃO']
print("datas ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

count_freq(df, 'data')
#print("Dias mensionados nas mensagens:")
#print(sorted(df['data'].unique()))

data probs: 5
datas ok: 260


,Freq,Perc
data,,
07/07,34,13.076923
14/07,23,8.846154
08/07,21,8.076923
05/07,19,7.307692
09/07,18,6.923077
02/07,18,6.923077
12/07,17,6.538462
04/07,16,6.153846
11/07,16,6.153846


# Unidades

In [12]:
df['unidade'] = ''
df['Q_unidades'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    
    #Unidades escrita com a bunda
    lista_avi = ["CENTRAL","AVICENA","AVICENNA","AVICCENNA"]
    for i_avi in lista_avi:
        mensagem = mensagem.replace(i_avi, 'AVICCENA')
        
    mensagem = mensagem.replace('Sta Isabel', 'SANTA')
    
    lista_unidades = re.findall(r'ALPHA|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VILA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades == 1: #capturando apenas com uma unidade
        df['unidade'][i] = lista_unidades[0].replace('  ',' ').upper().strip()
    elif quantidade_unidades > 1: #com mais de uma unidade
        df['unidade'][i] = lista_unidades
        df['controle'][i] = 'DESLOCADO'
    else: #sem nenhuma unidade
        df['controle'][i] = 'UNIDADES'
        
print("quantidades de unidades presentes nas mensagens:")
count_freq(df, 'Q_unidades')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'UNIDADES']])
print("unidades probs:", len(df_prob))
sample_prob('UNIDADES')

df = df[df['controle'] != 'UNIDADES']

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
#print("unidade ok:", len(df)) 
#print("unidade ok deslocado:", len(df[df['controle'] == 'DESLOCADO']))

quantidades de unidades presentes nas mensagens:


,Freq,Perc
Q_unidades,,
1,229,88.076923
0,16,6.153846
2,15,5.769231


unidades probs: 21


# Deslocados

In [13]:
#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DESLOCADO', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'DESLOCADO'
        
count_freq(df, 'controle')
# OBS: Plantões de deslocado não são mais excluido

,Freq,Perc
controle,,
DESLOCADO,15,100.0


# (Retirada) Distancia e Saude

In [14]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA
df['controle2'] = np.nan
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISTANCIA|D[0-9]', mensagem)
    quantidade_observacoes = len(observacoes)

    if quantidade_observacoes >= 1:
        df['controle'][i] = 'DISP_SAUDE_DIST'
        if quantidade_observacoes == 1:
            df['controle2'][i] = observacoes[0]
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISP_SAUDE_DIST']])
print("unidades probs:", len(df_prob))
sample_prob('DISP_SAUDE_DIST')

df = df[df['controle'] != 'DISP_SAUDE_DIST']
print("unidade ok:", len(df)) 

print(df_prob['mensagem'][df_prob['controle'] == 'DISP_SAUDE_DIST'].sample(2))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DISP_SAUDE_DIST,108,94.736842
DESLOCADO,6,5.263158


unidades probs: 129
unidade ok: 136
0      05/07 ANALIA  -  ALESSANDRE ENTRADA 07:00  ( D...
146    16/07 JABAQUARA VIVIANNA ENTRADA 07:00  ( DIST...
Name: mensagem, dtype: object


In [15]:
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall('D[0-9]|DISTANCIA', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'DISTANCIAS'

count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISTANCIAS']])
print("probs:", len(df_prob))
sample_prob('DISTANCIAS')

df = df[df['controle'] != 'DISTANCIAS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,6,100.0


probs: 129
ok: 136


# Extras

In [16]:
df["extra"] = np.nan

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se conter a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
print("Quantidade de registros Extra - Voluntario:")
count_freq(df, 'extra')      

Quantidade de registros Extra - Voluntario:


,Freq,Perc
extra,,
extra,2,100.0


# Tipo entrada e saida

In [17]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = np.nan

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'SEM_ENTRADA_OU_SAIDA'
    else: #restante com mais de uma informação
        df['tipo'][i] = entradas_saidas[0]
        
count_freq(df, 'tipo')
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'SEM_ENTRADA_OU_SAIDA']])
print("unidades probs:", len(df_prob))
sample_prob('SEM_ENTRADA_OU_SAIDA')

df = df[df['controle'] != 'SEM_ENTRADA_OU_SAIDA']
print("unidade ok:", len(df)) 

#print(df_prob['mensagem'][df_prob['controle'] == 'SEM_ENTRADA_OU_SAIDA'].sample(1))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

,Freq,Perc
tipo,,
SAIDA,86,66.666667
ENTRADA,43,33.333333


,Freq,Perc
controle,,
SEM_ENTRADA_OU_SAIDA,7,58.333333
DESLOCADO,5,41.666667


unidades probs: 136
unidade ok: 129
ok: 129


# -----------------------------------------

# Data

In [18]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = np.nan

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [19]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['03/07', '13/07', '05/07', '04/07', '01/07', '02/07', '08/07',
       '09/07', '07/07', '11/07', '15/07', '12/07', '06/07', '10/07',
       '16/07'], dtype=object)

In [20]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2024" 
    
count_freq(df, 'data')

,Freq,Perc
data,,
08/07/2024,16,12.403101
11/07/2024,12,9.302326
05/07/2024,11,8.527132
04/07/2024,11,8.527132
09/07/2024,11,8.527132
12/07/2024,11,8.527132
03/07/2024,10,7.751938
02/07/2024,9,6.976744
01/07/2024,8,6.201550


In [21]:
df['sem_ou_fds'] = 'semana'

for i in lista_fds:
    datinha = i+"/"+mes+"/2024"
    df['sem_ou_fds'][df['data'] == datinha] = 'fds'

count_freq(df, 'sem_ou_fds')

,Freq,Perc
sem_ou_fds,,
semana,115,89.147287
fds,14,10.852713


In [22]:
print(df.groupby(['data', 'sem_ou_fds']).size().unstack())

sem_ou_fds  fds  semana
data                   
01/07/2024  NaN     8.0
02/07/2024  NaN     9.0
03/07/2024  NaN    10.0
04/07/2024  NaN    11.0
05/07/2024  NaN    11.0
06/07/2024  2.0     NaN
07/07/2024  5.0     NaN
08/07/2024  NaN    16.0
09/07/2024  NaN    11.0
10/07/2024  NaN     8.0
11/07/2024  NaN    12.0
12/07/2024  NaN    11.0
13/07/2024  7.0     NaN
15/07/2024  NaN     6.0
16/07/2024  NaN     2.0


In [23]:
#df['data'].iloc[180] = '26/09/2022'

# Hora

In [24]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [25]:
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'HORA'
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORA']])
print("probs:", len(df_prob))
sample_prob('HORA')

df = df[df['controle'] != 'HORA']
print("ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
HORA,12,70.588235
DESLOCADO,5,29.411765


probs: 148
ok: 117


In [26]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['01:40', '01:35', '23:45', '19:00', '23:40', '05:30', '11:00',
       '07:00', '18:00', '17:20', '00:45', '00:15', '00:40', '16:18',
       '08:30', '18:50', '18:55', '18:40', '18:45', '18:37', '06:50',
       '13:10', '09:20', '06:30', '16:10', '05:40', '05:20', '17:55',
       '04:15', '17:40', '16:45', '06:00', '16:40', '09:05', '17:50',
       '15:00', '17:45', '17:10', '17:00', '04:25', '04:30', '04:20',
       '05:00', '10:50', '16:00', '15:40', '06:15', '09:45', '15:15',
       '18:20', '18:10', '16:30'], dtype=object)

# Hora final

In [27]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [28]:
df['hora'].unique()

array(['01:40', '01:35', '23:45', '19:00', '23:40', '05:30', '11:00',
       '07:00', '18:00', '17:20', '00:45', '00:15', '00:40', '16:18',
       '08:30', '18:50', '18:55', '18:40', '18:45', '18:37', '06:50',
       '13:10', '09:20', '06:30', '16:10', '05:40', '05:20', '17:55',
       '04:15', '17:40', '16:45', '06:00', '16:40', '09:05', '17:50',
       '15:00', '17:45', '17:10', '17:00', '04:25', '04:30', '04:20',
       '05:00', '10:50', '16:00', '15:40', '06:15', '09:45', '15:15',
       '18:20', '18:10', '16:30'], dtype=object)

In [29]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'Hora_final'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'Hora_final'
        else:
            df['controle'][i] = 'Hora_final'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'Hora_final'

df['hora'].unique()

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'Hora_final']])
print("probs:", len(df_prob))
sample_prob('Hora_final')

df = df[df['controle'] != 'Hora_final']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

[1900]
[1]
[1]
[ 1 23 19  5 11  7 18 17  0 16  8  6 13  9  4 15 10]
probs: 148
ok: 117


# Setor

In [30]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''

df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        else:
            df['controle'][i] = 'HORARIOS'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto >= 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            if df['hora'][i] == '13:00':
                df['SAIDA'][i] = '13:00'
                df['ENTRADA'][i] = '07:00'
                df['PERIODO'][i] = 'manha'
            else:
                df['ENTRADA'][i] = '13:00'
                df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'HORARIOS'
    else:
        df['controle'][i] = 'HORARIOS'

df['controle'][df['ENTRADA'] == df['SAIDA']] = 'HORARIOS'

count_freq(df, 'PERIODO')
count_freq(df, 'controle')

df['periodo'] = np.nan
df['periodo'][df['PERIODO'] == '6h'] = 1
df['periodo'][df['PERIODO'] == 'manha'] = 2
df['periodo'][df['PERIODO'] == 'tarde'] = 3
df['periodo'][df['PERIODO'] == 'noturno saida'] = 4

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORARIOS']])
print("probs:", len(df_prob))
sample_prob('HORARIOS')

df = df[df['controle'] != 'HORARIOS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
PERIODO,,
tarde,42,35.897436
manha,24,20.512821
6h,22,18.803419
,16,13.675214
noturno saida,13,11.111111


,Freq,Perc
controle,,
HORARIOS,41,89.130435
DESLOCADO,5,10.869565


probs: 189
ok: 76


# Duração do Plantão

In [31]:
df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'][df['DURAÇÃO'] == '00:00'] = '06:00'    

df['DURAÇÃO'].unique()

array(['01:30', '02:00', '06:00', '05:00', '04:30', '03:30', '00:30',
       '02:30', '03:00', '04:00', '01:00', '05:30'], dtype=object)

# Checagem Unidades:

In [32]:
# alterando JABA e AVVICENA
def replace_values(value):
    if value == 'JABA':
        return 'JABAQUARA'
    elif value == 'AVICCENA':
        return 'CENTRAL DO TATUAPE'
    elif value == 'SANTA':
        return 'SANTA ISABEL'
    else:
        return value
    
def replace_values2(value):
    if isinstance(value, list):
        return [replace_values(v) for v in value]
    else:
        return replace_values(value)

# Aplicar a função à coluna 'setor'
df['unidade'] = df['unidade'].apply(lambda x: replace_values2(x))

In [33]:
# Cria uma nova Série sem as listas
serie_sem_listas = pd.Series([x for x in df['unidade'] if not isinstance(x, list)])

# Imprime os valores únicos da nova Série
print(serie_sem_listas.unique())

['SANTA ISABEL' 'VILLA' 'ITAIM' 'ANALIA' 'JABAQUARA' 'VNS' 'OSASCO'
 'CENTRAL DO TATUAPE' 'MORUMBI']


In [34]:
# Cria uma lista com apenas os valores que são listas
listas = [x for x in df['unidade'] if isinstance(x, list)]

# Converte as listas em tuplas e adiciona elas a um conjunto para obter apenas as listas distintas
listas_distintas = set(tuple(lista) for lista in listas)

# Imprime as listas distintas
for lista in listas_distintas:
    print(list(lista))


['MORUMBI', 'ITAIM']
['ITAIM', 'VNS']
['OSASCO', 'ITAIM']


# Setor (2)

In [35]:
df['setor'] = np.nan

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'SEMANA - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'FDS - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
count_freq(df, 'setor')

,Freq,Perc
setor,,
SEMANA - TARDE - C.CIRURGICO,41,53.947368
SEMANA - 6HS,21,27.631579
SEMANA - MANHA - C.CIRURGICO,8,10.526316
FDS - C.CIRURGICO ROTINA,4,5.263158
FDS - SAIDA,1,1.315789
FDS - 6HS,1,1.315789


# *****************************************

# Pareamento

In [36]:
print(len(df))
print(len(df_prob))

76
189


In [37]:
df_prob_pareado = pd.DataFrame(columns=df.columns)
df_prob_pareado

,controle,chat,dia,nome,mensagem,data,unidade,Q_unidades,controle2,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,periodo,DURAÇÃO,setor


In [38]:
count_ok = 0
count_erro = 0

for nome in sorted(df['nome'].unique()):
    df_filter = df[df['nome'] == nome]
    for data in sorted(df_filter['data'].unique()):
        df_filter = df[(df['nome'] == nome) & (df['data'] == data)]

        if len(df_filter) >= 2:
            qtd_entrada = len(df_filter[df_filter['tipo'] == 'ENTRADA'])
            qtd_saida = len(df_filter[df_filter['tipo'] == 'SAIDA'])

            if qtd_entrada == qtd_saida:
                print(nome, '|', data)
                count_ok += 1
        else:
            # Procurar se o registro foi mandado com o erro:
            df_prob_filter = df_prob[(df_prob['nome'] == nome) & (df_prob['data'] == data)]

            # ERRO PARCIAL
            if len(df_prob_filter) > 0: 
                print(nome, '|', data, '|', 'PARCIAL')
                df_prob['controle'][df_prob_filter.index] = 'erro_parcial'
                
            # ERRO DE PERDA TOTAL
            else: 
                print(nome, '|', data, '|', 'ERRO')
                df_filter['controle'] = 'erro_total'
                df_prob_pareado = pd.concat([df_prob_pareado, df_filter])

            count_erro += 1
    print('='*50)

AMANDA KIYOKO SOUZA KOSSAKA | 06/07/2024 | ERRO
AMANDA KIYOKO SOUZA KOSSAKA | 09/07/2024 | ERRO
ANA CLARA MOURAO BARRETO | 10/07/2024 | ERRO
ANDRE AUGUSTO BELLINI | 12/07/2024 | ERRO
ANDRE DA COSTA AGUIAR VENTURA | 05/07/2024 | ERRO
ANNA BEATRIZ ARANHA TREVISAN | 02/07/2024 | ERRO
ANNA BEATRIZ ARANHA TREVISAN | 03/07/2024 | ERRO
ANNA BEATRIZ ARANHA TREVISAN | 09/07/2024 | ERRO
ARTHUR OLIVEIRA FACCHINI | 11/07/2024 | ERRO
ARTHUR RODRIGUES TORRELIO | 03/07/2024 | ERRO
AUGUSTO DE CASTRO SANTOS QUEIROZ | 08/07/2024 | ERRO
BEATRIZ ANTIPOU DOS SANTOS | 15/07/2024 | ERRO
BRIAN SOSSAI PACHECO | 08/07/2024 | ERRO
BRUNA DAIBERT DANESIN | 05/07/2024 | ERRO
CAIO TULIO DUARTE | 03/07/2024 | ERRO
CAIO TULIO DUARTE | 10/07/2024 | ERRO
CAMILA RESENDE DE PAULA | 11/07/2024 | ERRO
DANIEL MASUZAKI WONG | 05/07/2024 | ERRO
DANIEL MASUZAKI WONG | 12/07/2024 | ERRO
EDUARDO ARBACHE BEZERRA | 09/07/2024
ESTANRLEY BARCELOS PINTO | 11/07/2024 | ERRO
FELIPE DE OLIVEIRA NUNES | 11/07/2024 | ERRO
FERNANDO AKIRA KO

# (0) Salvando problemas de entrada e saida

In [39]:
print(count_ok)
print(count_erro)

2
72


In [40]:
salvando_nome_arquivo('df_entrada_saida.xlsx', df_prob_pareado)

70


# Ultimos ajustes:

In [41]:
df = df[['chat', 'dia', 'nome', 'mensagem', 'controle', 'data', 'unidade',
       'extra', 'tipo', 'ENTRADA', 'SAIDA', 'DURAÇÃO', 'PERIODO','periodo',
         'setor', 'sem_ou_fds']]

# =========================================

# Deslocados:

In [42]:
df['controle'][(df['PERIODO'] == '6h') & (df['controle'] == 'DESLOCADO')] = 'DESLOCADO 6H'
count_freq(df, 'controle')

,Freq,Perc
controle,,
DESLOCADO 6H,3,60.0
DESLOCADO,2,40.0


In [43]:
df['unidade2'] = df['unidade']

for i in range(0, len(df)):
    if isinstance(df.loc[i, 'unidade'], list):
        df['unidade'][i] = df['unidade'][i][0]

df['unidade'][df['unidade'] == 'VILA'] = 'VILLA'
df['unidade'][df['unidade'] == 'JABA'] = 'JABAQUARA'
df['unidade'][df['unidade'] == 'SANTA'] = 'SANTA ISABEL'

sorted(df['unidade'].unique())
count_freq(df, 'unidade')

,Freq,Perc
unidade,,
JABAQUARA,16,21.052632
ITAIM,13,17.105263
VILLA,12,15.789474
OSASCO,11,14.473684
MORUMBI,8,10.526316
ANALIA,7,9.210526
VNS,7,9.210526
SANTA ISABEL,1,1.315789
CENTRAL DO TATUAPE,1,1.315789


# (1) Salvando tudo: gabs & giov

In [44]:
salvando_nome_arquivo('df_gabs.xlsx', df)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

76
.
189


# =========================================

# Dispensados:

In [45]:
df = pd.read_excel('df_giov.xlsx')[['chat','dia','nome','mensagem']]
df = df.drop_duplicates(subset=['chat'])
df = df.sort_values(['nome', 'dia'])
print(len(df))
df.sample(3)

189


,chat,dia,nome,mensagem
170,[05/07/2024 20:06:57] Beatriz Antipou dos Sant...,05/07/2024,BEATRIZ ANTIPOU DOS SANTOS,05/07 - VILLA - ANTIPOU - SAÍDA 18:40
10,[10/07/2024 08:44:04] Leonardo Barbosa Santos:...,10/07/2024,LEONARDO BARBOSA SANTOS,10/07 HCT L.SANTOS ENTRADA 7:00 ( PLANTÃO )
142,[02/07/2024 06:49:39] Fernanda Teles Ribeiro: ...,02/07/2024,FERNANDA TELES RIBEIRO,02/07 ITAIM - FERNANDA ENTRADA 06H


In [46]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA

df['controle'] = np.nan

#preparando para reconhecer corretamente

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISPENSADA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes == 0:
        df['controle'][i] = 'PROB'
        
df['controle'].value_counts() 

#salvando
df_prob = df[df['controle'] == 'PROB']
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

unidades probs: 186
unidade ok: 3


In [47]:
#DATA DO PLANTÃO
df['controle'] = ''
df['data'] = ''

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
        df['controle'][i] = 'NORM'
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'PROB'

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']])
df_prob.reset_index(drop=True, inplace=True)
print("data probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("datas ok:", len(df)) 

df['data'] = df['data']+"/2024"
sorted(df['data'].unique())

data probs: 186
datas ok: 3


['01/07/2024', '08/07/2024', '13/07/2024']

In [48]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio
    
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

#Retirando aqueles que contem horas no registro da mensagem
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas >= 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
        df['controle'][i] = 'PROB'
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'OK'
        
print(df['controle'].value_counts())

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'OK']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] == 'OK']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

controle
PROB    2
OK      1
Name: count, dtype: int64
probs: 188
ok: 1


In [49]:
print(len(df_prob))
print(len(df))

188
1


# (2) Salvando tudo: giov & dispensados

In [50]:
df = df.drop_duplicates(subset=['chat'])

os.remove("df_giov.xlsx")
salvando_nome_arquivo('df_giov.xlsx', df_prob)
print('.')
salvando_nome_arquivo('df_dispensados.xlsx', df)

188
.
1


# ===========================================

# Ultima Contagem

In [51]:
print(len(pd.DataFrame({'chat': read_file('_chat.txt') })))
print('.')
print(len(pd.read_excel('df_dispensados.xlsx')))
print('.')
print(len(pd.read_excel('df_gabs.xlsx')))
print('.')
print(len(pd.read_excel('df_giov.xlsx')))

3121
.
1
.
76
.
188
